In [45]:
import numpy as np
import torch
import torch.nn.functional as F
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline

In [9]:
classifier = pipeline('sentiment-analysis')
res = classifier('nigga you are my homie')
res

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9843875169754028}]

In [42]:
model_name = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
res = classifier('nigga you are my homie')
res

[{'label': 'POSITIVE', 'score': 0.9843875169754028}]

In [46]:
sequence = 'писька'

res = tokenizer(sequence)
print(res)

tokens = tokenizer.tokenize(sequence)
print(tokens)

encoding = tokenizer.encode(sequence)
print(encoding)

ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

decoded_string = tokenizer.decode(ids)
print(decoded_string)

{'input_ids': [101, 1194, 10325, 29747, 23742, 28598, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
['п', '##и', '##с', '##ь', '##ка']
[101, 1194, 10325, 29747, 23742, 28598, 102]
[1194, 10325, 29747, 23742, 28598]
писька


In [50]:
X_train = ["I've been waiting for a HuggingFace course my whole life.",
           "Python is great!"]

res = classifier(X_train)
print(res)

batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt")
print(batch)

with torch.no_grad():
    outputs = model(**batch)
    print(outputs)
    logits = outputs.logits
    print(logits)
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)

[{'label': 'POSITIVE', 'score': 0.9598049521446228}, {'label': 'POSITIVE', 'score': 0.9998615980148315}]
{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101, 18750,  2003,  2307,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-4.2745,  4.6111]]), hidden_states=None, attentions=None)
tensor([[-1.5607,  1.6123],
        [-4.2745,  4.6111]])
tensor([[4.0195e-02, 9.5980e-01],
        [1.3835e-04, 9.9986e-01]])
tensor([1, 1])


In [36]:
X_train = ["I've been waiting for a HuggingFace course my whole life.",
           "Python is great!"]

res = classifier(X_train)
print(res)

batch = tokenizer(X_train,
                  padding=True,  # добавление нулевых значений для достижения max(current_length,max_length) 
                  truncation=True,  # укорочение вектора из токенов до max_length
                  max_length=512,
                  return_tensors="pt")
print(batch)

with torch.no_grad():
    outputs = model(**batch)
    print(outputs)
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)

[{'label': 'POSITIVE', 'score': 0.9598049521446228}, {'label': 'POSITIVE', 'score': 0.9998615980148315}]
{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101, 18750,  2003,  2307,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-4.2745,  4.6111]]), hidden_states=None, attentions=None)
tensor([[4.0195e-02, 9.5980e-01],
        [1.3835e-04, 9.9986e-01]])
tensor([1, 1])


Fine-tune

In [64]:
dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors='pt')


tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle().select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle().select(range(100))

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [67]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased",
                                                           num_labels=5,
                                                           torch_dtype="auto")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print(labels, predictions)
    return np.mean(predictions == labels)


training_args = TrainingArguments(output_dir="test_trainer", 
                                  eval_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [3]:
import gensim.downloader
model = gensim.downloader.load("glove-wiki-gigaword-50")

In [18]:
model["sa"]

array([ 0.44698 , -0.82866 ,  0.13248 ,  1.6964  , -0.1525  , -0.82394 ,
        0.12727 ,  0.12931 , -0.54606 ,  1.4037  ,  1.8477  ,  0.24397 ,
       -0.55859 , -0.87021 , -0.80945 , -0.41357 , -0.87649 , -0.13215 ,
       -0.01303 , -0.23819 ,  0.20622 , -0.52825 , -0.095328, -0.16396 ,
       -1.8154  , -0.40307 , -1.3003  , -0.27408 , -0.43607 , -0.28226 ,
        1.6663  ,  0.65489 ,  0.011497,  1.8409  ,  0.2311  , -1.6303  ,
       -0.83408 ,  0.035609,  1.3365  ,  0.41262 ,  0.81518 ,  0.1347  ,
        0.78877 , -1.3599  , -0.33647 , -0.55555 , -1.0205  , -0.10541 ,
        1.1815  , -0.010285], dtype=float32)